<a href="https://colab.research.google.com/github/enzoampil/tito-joker/blob/master/experiments/Jokes_GPT2_Finetuning2_2_sentiment_controls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tito Joker v2.2

## Updates:
1. Sentiment will be infused into the jokes dataset w/ a special sentiment token
3. Sentiment controls to be added to the webapp

## Note that the experiment notebooks have ending numbers corresponding to the model's number (this is model2)

In [2]:
!pip install transformers

     |████████████████████████████████| 573kB 5.8MB/s 
     |████████████████████████████████| 870kB 15.6MB/s 
     |████████████████████████████████| 3.7MB 14.6MB/s 
     |████████████████████████████████| 1.0MB 35.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=405e5d147d40782da6c7342337e936c861a4a39de06e2517f97a97789707846f
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [3]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 23645 (delta 34), reused 34 (delta 16), pack-reused 23570
Receiving objects: 100% (23645/23645), 14.20 MiB | 13.69 MiB/s, done.
Resolving deltas: 100% (16729/16729), done.


In [4]:
!git clone https://github.com/enzoampil/tito-joker.git

Cloning into 'tito-joker'...
remote: Enumerating objects: 154, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 507 (delta 85), reused 130 (delta 67), pack-reused 353
Receiving objects: 100% (507/507), 6.13 MiB | 9.33 MiB/s, done.
Resolving deltas: 100% (281/281), done.


## Download raw jokes dataset

In [5]:
!wget https://storage.googleapis.com/joke-generator-model1/short-jokes.zip && unzip short-jokes.zip

--2020-04-09 09:12:50--  https://storage.googleapis.com/joke-generator-model1/short-jokes.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.126.128, 2a00:1450:4013:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.126.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10679873 (10M) [application/zip]
Saving to: ‘short-jokes.zip’

short-jokes.zip     100%[===================>]  10.18M  33.9MB/s    in 0.3s    

2020-04-09 09:12:51 (33.9 MB/s) - ‘short-jokes.zip’ saved [10679873/10679873]

Archive:  short-jokes.zip
  inflating: shortjokes.csv          


## Process the raw jokes

In [0]:
import pandas as pd

In [6]:
%cd tito-joker

/content/tito-joker


## Prepare dataset

In [0]:
# Won't run this first since sentiment analysis isn't applied here yet
#!python ./src/utils/process_jokes.py ../shortjokes.csv ./data/riddle_jokes.txt

In [0]:
def process_jokes(raw_fp):
    """
    Returns jokes dataframe w/ no special tokens yet.
    
    This makes sense in this context since we're using a pretrained sentiment analysis model which doesn't recognize our new joke speciic tokens.
    """
    df = pd.read_csv(raw_fp)

    # Append token at the end of each joke to indicate the end of a joke

    what_jokes = df[df.Joke.str.lower().str.startswith("what")].Joke.str.split("?")
    how_jokes = df[df.Joke.str.lower().str.startswith("how")].Joke.str.split("?")
    why_jokes = df[df.Joke.str.lower().str.startswith("why")].Joke.str.split("?")
    when_jokes = df[df.Joke.str.lower().str.startswith("when")].Joke.str.split("?")
    where_jokes = df[df.Joke.str.lower().str.startswith("where")].Joke.str.split("?")

    jokes = []
    for joke_ in [what_jokes, how_jokes, why_jokes, when_jokes, where_jokes]:
        joke_df_ = pd.DataFrame(joke_.values.tolist()).iloc[:, :2].dropna()
        joke_df_.columns = ["question", "answer"]
        jokes.append(joke_df_)

    jokes_df = pd.concat(jokes)
    jokes_df = (
        jokes_df[~(jokes_df.answer.isin([""]))].drop_duplicates().reset_index(drop=True)
    )

    return jokes_df

In [0]:
jokes_df = process_jokes('../shortjokes.csv')

In [0]:
jokes_df['question_answer'] = (jokes_df.question + "? " + jokes_df.answer)

In [42]:
jokes_df

,question,answer,question_answer
0,What do you do if a bird shits on your car,Don't ask her out again.,What do you do if a bird shits on your car? D...
1,What should you do before criticizing Pac-Man,WAKA WAKA WAKA mile in his shoes,What should you do before criticizing Pac-Man?...
2,What's the difference between an illegal Mexic...,Nothing... they were both made to steal Ameri...,What's the difference between an illegal Mexic...
3,What do you call a barbarian you can't see,an Invisigoth.,What do you call a barbarian you can't see? a...
4,What did Arnold Schwarzenegger say at the abor...,"Hasta last vista, baby.",What did Arnold Schwarzenegger say at the abor...
...,...,...,...
66638,Where does dubious pasta come from,"The spaghetto. I can't take all the credit, h...",Where does dubious pasta come from? The spagh...
66639,Where does the midget pizza chef with epilepsy...,Little seizures,Where does the midget pizza chef with epilepsy...
66640,Where do you hide an airport,IN PLANE SIGHT!,Where do you hide an airport? IN PLANE SIGHT!
66641,Where did L Ron Hubbard store his dishes,In the L Ron cupboard.,Where did L Ron Hubbard store his dishes? In ...


# Run sentiment analysis on the jokes

## There are 3 approaches we can take in scoring sentiment
1. **Score both question and answer**
2. Score only the question
3. **Score only the answer**

*Since in Tito Joker's question, we expect users to ask the question, I believe it makes more sense to score the answer specifically. However, I also think that answers are quite short and may not give enough context. To be safe, I will apply both 1 and 3.*

I also realized just now that 1 will be the easiest to implement as a special token, since we can just play the tag right after the CLS token. For 3, it will be harder if we want to add the token after the question at inference. It's definitely doable but we will have to run experiments.

With the answer only implementation, I might still just put the token at the beginning of the model. The problem is that the question will also be dependendt on the sentiment tag, but this becomes less of an issue as long as people actually ask questions.

In [0]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
from tqdm import tqdm


In [49]:
import torch
torch.cuda.current_device()

0

In [50]:
nlp = pipeline('sentiment-analysis', device=0)

In [51]:
#Test
nlp("I am very glad to be here")

[{'label': 'POSITIVE', 'score': 0.9996952}]

In [0]:
answer_sentiment = [nlp(ans) for ans in jokes_df.answer.values[:20]]

In [30]:
# The results look mixed. The long answers seem to have accurate answers, but the one word answers don't seem right
for i in range(20):
    print(answer_sentiment[i], jokes_df.answer.values[i])

[{'label': 'NEGATIVE', 'score': 0.98510593}]  Don't ask her out again.
[{'label': 'POSITIVE', 'score': 0.93220097}]  WAKA WAKA WAKA mile in his shoes
[{'label': 'NEGATIVE', 'score': 0.9994851}]  Nothing... they were both made to steal American jobs.
[{'label': 'POSITIVE', 'score': 0.77296317}]  an Invisigoth.
[{'label': 'POSITIVE', 'score': 0.9947985}]  Hasta last vista, baby.
[{'label': 'POSITIVE', 'score': 0.61819375}]  Spudnik
[{'label': 'POSITIVE', 'score': 0.99789554}]  Reserection
[{'label': 'POSITIVE', 'score': 0.99252135}]  The New York jets.
[{'label': 'POSITIVE', 'score': 0.9998734}]  One's a Goodyear, an the other's a great year.
[{'label': 'NEGATIVE', 'score': 0.99524903}]  When you dump your load in a washer, it doesn't follow you around for a week.
[{'label': 'POSITIVE', 'score': 0.99945635}]  Stevie Wonder
[{'label': 'NEGATIVE', 'score': 0.9496889}]  I've been through a lot.
[{'label': 'POSITIVE', 'score': 0.9976123}]  Make sure to come upstairs.
[{'label': 'NEGATIVE', '

In [0]:
question_sentiment = [nlp(ans) for ans in jokes_df.question.values[:20]]

In [37]:
# I find the results here not so convincing
for i in range(20):
    print(question_sentiment[i], jokes_df.question.values[i])

[{'label': 'NEGATIVE', 'score': 0.9979136}] What do you do if a bird shits on your car
[{'label': 'POSITIVE', 'score': 0.7655616}] What should you do before criticizing Pac-Man
[{'label': 'POSITIVE', 'score': 0.8212372}] What's the difference between an illegal Mexican and an autonomous robot...
[{'label': 'NEGATIVE', 'score': 0.94881314}] What do you call a barbarian you can't see
[{'label': 'NEGATIVE', 'score': 0.9652808}] What did Arnold Schwarzenegger say at the abortion clinic
[{'label': 'NEGATIVE', 'score': 0.9907236}] What do you call a potato in space
[{'label': 'NEGATIVE', 'score': 0.99344915}] What happens to a necrophiliac after death
[{'label': 'POSITIVE', 'score': 0.99935293}] What's Al-Qaeda's favorite American football team
[{'label': 'NEGATIVE', 'score': 0.80405384}] What's the difference between a car tyre, and 365 condoms
[{'label': 'POSITIVE', 'score': 0.8771809}] What's the difference between a blonde and a washer
[{'label': 'NEGATIVE', 'score': 0.99969804}] What's 

In [0]:
question_answer_sentiment = [nlp(ans) for ans in jokes_df.question_answer.values[:20]]

In [44]:
# This one is sort of in the middle
for i in range(20):
    print(question_answer_sentiment[i], jokes_df.question_answer.values[i])

[{'label': 'NEGATIVE', 'score': 0.99840784}] What do you do if a bird shits on your car?  Don't ask her out again.
[{'label': 'NEGATIVE', 'score': 0.9913214}] What should you do before criticizing Pac-Man?  WAKA WAKA WAKA mile in his shoes
[{'label': 'NEGATIVE', 'score': 0.99952585}] What's the difference between an illegal Mexican and an autonomous robot...?  Nothing... they were both made to steal American jobs.
[{'label': 'NEGATIVE', 'score': 0.5165516}] What do you call a barbarian you can't see?  an Invisigoth.
[{'label': 'NEGATIVE', 'score': 0.98074615}] What did Arnold Schwarzenegger say at the abortion clinic?  Hasta last vista, baby.
[{'label': 'NEGATIVE', 'score': 0.99436504}] What do you call a potato in space?  Spudnik
[{'label': 'NEGATIVE', 'score': 0.9864824}] What happens to a necrophiliac after death?  Reserection
[{'label': 'NEGATIVE', 'score': 0.8593313}] What's Al-Qaeda's favorite American football team?  The New York jets.
[{'label': 'POSITIVE', 'score': 0.9965733}]

My view so far is to go with answer sentiment tagging first

In [0]:
answer_sentiment_all = [nlp(ans) for ans in tqdm(jokes_df.answer.values)]

In [0]:
answer_sentiment_all_dict = [pred[0] for pred in answer_sentiment_all]

In [59]:
answer_sentiment_all_dict[0]

{'label': 'NEGATIVE', 'score': 0.9851059}

In [0]:
answer_sentiment_all_df = pd.DataFrame(answer_sentiment_all_dict)

In [61]:
answer_sentiment_all_df.head()

,label,score
0,NEGATIVE,0.985106
1,POSITIVE,0.932200
2,NEGATIVE,0.999485
3,POSITIVE,0.772967
4,POSITIVE,0.994798


In [0]:
jokes_sentiment_df = pd.concat([jokes_df, answer_sentiment_all_df], axis=1)

In [65]:
jokes_sentiment_df.head()

,question,answer,question_answer,label,score
0,What do you do if a bird shits on your car,Don't ask her out again.,What do you do if a bird shits on your car? D...,NEGATIVE,0.985106
1,What should you do before criticizing Pac-Man,WAKA WAKA WAKA mile in his shoes,What should you do before criticizing Pac-Man?...,POSITIVE,0.932200
2,What's the difference between an illegal Mexic...,Nothing... they were both made to steal Ameri...,What's the difference between an illegal Mexic...,NEGATIVE,0.999485
3,What do you call a barbarian you can't see,an Invisigoth.,What do you call a barbarian you can't see? a...,POSITIVE,0.772967
4,What did Arnold Schwarzenegger say at the abor...,"Hasta last vista, baby.",What did Arnold Schwarzenegger say at the abor...,POSITIVE,0.994798


In [0]:
#jokes_sentiment_df.to_csv('riddle_jokes_answer_sentiment.csv')

In [5]:
ls sample_data

anscombe.json*                mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md*


In [0]:
from google.colab import auth
auth.authenticate_user()

In [70]:
#!gsutil cp riddle_jokes_answer_sentiment.csv gs://joke-generator-model1/

Copying file://riddle_jokes_answer_sentiment.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/12.3 MiB.                                     


## Convert riddle df to string

In [0]:
def riddle_df_to_str(df, target_fp):
    sentiment_tags = df.label.apply(lambda x: '<positive>' if x == 'POSITIVE' else '<negative>')
    riddle_jokes_list = (
        "<soq> " + sentiment_tags + " " + df.question + " <eoq> " + df.answer + " <|endoftext|>"
    ).values.tolist()
    riddle_jokes = "\n".join(riddle_jokes_list)

    with open(target_fp, "w") as f:
        f.write(riddle_jokes)

In [7]:
!gsutil cp gs://joke-generator-model1/riddle_jokes_answer_sentiment.csv riddle_jokes_answer_sentiment.csv

Copying gs://joke-generator-model1/riddle_jokes_answer_sentiment.csv...
- [1 files][ 12.3 MiB/ 12.3 MiB]                                                
Operation completed over 1 objects/12.3 MiB.                                     


In [0]:
import pandas as pd
jokes_sentiment_df = pd.read_csv('riddle_jokes_answer_sentiment.csv')

In [9]:
riddle_str = riddle_df_to_str(jokes_sentiment_df, 'riddle_jokes_answer_sentiment.txt')

NameError: ignored

In [18]:
# Looks good
!head -5 riddle_jokes_answer_sentiment.txt

<soq> <negative> What do you do if a bird shits on your car <eoq>  Don't ask her out again. <|endoftext|>
<soq> <positive> What should you do before criticizing Pac-Man <eoq>  WAKA WAKA WAKA mile in his shoes <|endoftext|>
<soq> <negative> What's the difference between an illegal Mexican and an autonomous robot... <eoq>  Nothing... they were both made to steal American jobs. <|endoftext|>
<soq> <positive> What do you call a barbarian you can't see <eoq>  an Invisigoth. <|endoftext|>
<soq> <positive> What did Arnold Schwarzenegger say at the abortion clinic <eoq>  Hasta last vista, baby. <|endoftext|>


In [10]:
!gsutil cp riddle_jokes_answer_sentiment.txt gs://joke-generator-model1/riddle_jokes_answer_sentiment_20200409.txt

CommandException: No URLs matched: riddle_jokes_answer_sentiment.txt


## Save tokenizer w/ new tokens

In [11]:
!gsutil cp gs://joke-generator-model1/riddle_jokes_answer_sentiment_20200409.txt riddle_jokes_answer_sentiment.txt

Copying gs://joke-generator-model1/riddle_jokes_answer_sentiment_20200409.txt...
- [1 files][  7.4 MiB/  7.4 MiB]                                                
Operation completed over 1 objects/7.4 MiB.                                      


In [0]:
from transformers import GPT2Model, GPT2Config

In [0]:
#from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
#from pathlib import Path

In [0]:
!mkdir models
!mkdir models/pretrained
!mkdir models/finetuned

In [15]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# Adding the sentiment tokens as additional tokens
new_tokens = ['<soq>', '<eoq>', '<negative>', '<positive>']
num_added_toks = tokenizer.add_tokens(new_tokens)

In [16]:
tokenizer._convert_token_to_id("<|endoftext|>")

50256

In [17]:
#for token in new_tokens:
#    print(tokenizer._convert_token_to_id(token))
num_added_toks

4

In [18]:
tokenizer.save_pretrained('./models/pretrained')

('./models/pretrained/vocab.json',
 './models/pretrained/merges.txt',
 './models/pretrained/special_tokens_map.json',
 './models/pretrained/added_tokens.json')

In [19]:
# New tokens are stored in added_tokens.json
cat ./models/pretrained/added_tokens.json

{"<soq>": 50257, "<eoq>": 50258, "<negative>": 50259, "<positive>": 50260}

In [29]:
!head -5 ./riddle_jokes_answer_sentiment.txt

<soq> <negative> What do you do if a bird shits on your car <eoq>  Don't ask her out again. <|endoftext|>
<soq> <positive> What should you do before criticizing Pac-Man <eoq>  WAKA WAKA WAKA mile in his shoes <|endoftext|>
<soq> <negative> What's the difference between an illegal Mexican and an autonomous robot... <eoq>  Nothing... they were both made to steal American jobs. <|endoftext|>
<soq> <positive> What do you call a barbarian you can't see <eoq>  an Invisigoth. <|endoftext|>
<soq> <positive> What did Arnold Schwarzenegger say at the abortion clinic <eoq>  Hasta last vista, baby. <|endoftext|>


In [0]:
import json
with open('./models/pretrained/config.json', 'w') as f:
    json.dump(GPT2Config().to_dict(), f)

## Finetune the GPT2 model

In [30]:
!python transformers/examples/run_language_modeling.py\
    --overwrite_output_dir\
    --output_dir=./models/finetuned \
    --model_type=gpt2\
    --model_name_or_path=gpt2\
    --tokenizer_name ./models/pretrained \
    --do_train\
    --train_data_file=./riddle_jokes_answer_sentiment.txt \
    --per_gpu_train_batch_size 10 \
    --block_size 100 # Set this arbitrarily for now. But most jokes will be within this range. We don't want to weight super long jokes anyway.\

04/09/2020 12:13:34 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/09/2020 12:13:34 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.699bbd1c449e9861456f359d6daa51bd523ac085b4b531ab0aad5a55d091e942
04/09/2020 12:13:34 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "_num_labels": 2,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 50256,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_dec

## Test results

In [22]:
%cd /content

/content


In [32]:
!python transformers/examples/run_generation.py \
    --model_type=gpt2 \
    --model_name_or_path=models/finetuned/ \
    --length=50 \
    --stop_token='<|endoftext|>' \
    --num_return_sequences 20

04/09/2020 12:15:53 - INFO - transformers.tokenization_utils -   Model name 'models/finetuned/' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'models/finetuned/' is a path, a model identifier, or url to a directory containing tokenizer files.
04/09/2020 12:15:53 - INFO - transformers.tokenization_utils -   loading file models/finetuned/vocab.json
04/09/2020 12:15:53 - INFO - transformers.tokenization_utils -   loading file models/finetuned/merges.txt
04/09/2020 12:15:53 - INFO - transformers.tokenization_utils -   loading file models/finetuned/added_tokens.json
04/09/2020 12:15:53 - INFO - transformers.tokenization_utils -   loading file models/finetuned/special_tokens_map.json
04/09/2020 12:15:53 - INFO - transformers.tokenization_utils -   loading file models/finetuned/tokenizer_config.json
04/09/2020 12:15:53 - INFO - transformers.configuration_utils -   loading configuration file models/finetuned/config.json
04/09/2020 12:15:53

## Save results to gcs

In [33]:
%cd models/finetuned 

/content/models/finetuned


In [34]:
ls

added_tokens.json  checkpoint-500/  pytorch_model.bin        training_args.bin
checkpoint-1000/   config.json      special_tokens_map.json  vocab.json
checkpoint-1500/   merges.txt       tokenizer_config.json


In [0]:
mkdir model2-2

In [0]:
mv added_tokens.json config.json merges.txt pytorch_model.bin special_tokens_map.json tokenizer_config.json training_args.bin vocab.json model2-2

In [37]:
!zip -r model2-2.zip  model2-2

  adding: model2-2/ (stored 0%)
  adding: model2-2/vocab.json (deflated 63%)
  adding: model2-2/training_args.bin (deflated 36%)
  adding: model2-2/added_tokens.json (deflated 35%)
  adding: model2-2/pytorch_model.bin (deflated 16%)
  adding: model2-2/special_tokens_map.json (deflated 52%)
  adding: model2-2/merges.txt (deflated 53%)
  adding: model2-2/tokenizer_config.json (deflated 48%)
  adding: model2-2/config.json (deflated 57%)


In [38]:
ls -l

total 452248
drwxr-xr-x 2 root root      4096 Apr  9 12:03 checkpoint-1000/
drwxr-xr-x 2 root root      4096 Apr  9 12:05 checkpoint-1500/
drwxr-xr-x 2 root root      4096 Apr  9 12:02 checkpoint-500/
drwxr-xr-x 2 root root      4096 Apr  9 12:17 model2-2/
-rw-r--r-- 1 root root 463084808 Apr  9 12:18 model2-2.zip


In [0]:
from google.colab import auth
auth.authenticate_user()

In [39]:
!gsutil cp model2-2.zip gs://joke-generator-model1/

Copying file://model2-2.zip [Content-Type=application/zip]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/441.6 MiB.                                    
